In [2]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import os
import sys

# Collaborative filtering

In [36]:
PROCESS_DATA = False

if PROCESS_DATA:
    is_first = True
    for i in range(1, 7):
        products_folder = f"sbermarket_tab_2_{i}"
        for file in os.listdir(products_folder):
            products_file = os.path.join(products_folder, file)
            if is_first:
                products = pd.read_csv(products_file, usecols=["user_id", "product_id", "quantity"])
                is_first = False
            else:
                new_products = pd.read_csv(products_file, usecols=["user_id", "product_id", "quantity"])
                products = pd.concat([products, new_products])

    products.rename(columns={"quantity": "purchase_count",
                             "user_id": "customerId",
                             "product_id": "productId"}, inplace=True)

    data = products.groupby(["customerId", "productId"]).agg({"purchase_count": "sum"})
    del products
    data.reset_index(inplace=True)
    data.to_csv("data.csv")

else:
    data = pd.read_csv("data.csv", index_col=0)
    print(f"Size  of the dataframe is {sys.getsizeof(data) / 1024**3:.1f} GB")
    print(f"Shape of the dataframe is {data.shape}")

/home/egor/.local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Size  of the dataframe is 1.4 GB
Shape of the dataframe is (46194236, 3)


In [37]:
data.head()

,customerId,productId,purchase_count
0,51,0,9
1,51,159,1
2,51,397,2
3,51,407,16
4,51,456,20


In [6]:
sample_submission = "sample_submission.csv"
customers = pd.read_csv(sample_submission, usecols=["Id"])
users_to_recommend = customers.Id.tolist()

user_id = 'customerId'
item_id = 'productId'
n_rec = 50  # number of items to recommend

In [5]:
FREQ_TYPE = 'scaled_purchase_freq'

if FREQ_TYPE == 'dummy':
    target = 'purchase_dummy'
    data['purchase_dummy'] = 1
elif FREQ_TYPE == 'rel_freq':
    target = 'scaled_purchase_freq'
    data["max_count_prod"] = data.groupby('productId')["purchase_count"].transform('max')
    data["scaled_purchase_freq"] = data["purchase_count"] / data["max_count_prod"]
    data.drop(columns=['max_count_prod', 'purchase_count'], inplace=True)
else:
    target = 'purchase_count'

train_data = tc.SFrame(data)
del data

In [6]:
def create_output(df_rec, name):
    df_rec['Predicted'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: ' '.join(x.astype(str)))
    df_rec.rename(columns={"customerId": "Id"}, inplace=True)
    df_output = df_rec[['Id', 'Predicted']].drop_duplicates() \
        .sort_values('Id').set_index('Id')
    df_output.to_csv(f'submission_{name}_{FREQ_TYPE}.csv')
    return df_output

def fit_predict(train_data, name, user_id, item_id, target, users_to_recommend, n_rec):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec).to_dataframe()
    return create_output(recom, name)

In [7]:
SIMILARITY = 'cosine'
submission = fit_predict(train_data, SIMILARITY, user_id, item_id, target, users_to_recommend, n_rec)

Preparing data set.

Data has 46194236 observations with 657502 users and 95002 items.

Data prepared in: 22.6766s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 103000/107068 queries. users per second: 1721.85

recommendations finished on 104000/107068 queries. users per second: 1722.82

recommendations finished on 105000/107068 queries. users per second: 1723.45

recommendations finished on 106000/107068 queries. users per second: 1720.04

recommendations finished on 107000/107068 queries. users per second: 1706.99

In [10]:
submission.head()

(107068, 2)


,Id,Predicted
0,51,5074743 99474 86371 7342276 27555 5061861 3819...
1,65,5469728 5479511 94333 55133 100812 69669 54816...
2,187,88085 88095 7407972 3302679 9220725 88889 8949...
3,400,709 100789 69669 100849 55133 63072 3817542 34...
4,576,10526485 10527160 85433 4275615 13577 10527066...


# Recommend items that user buys frequently

In [4]:
is_first = True
for i in range(1, 7):
    products_folder = f"sbermarket_tab_2_{i}"
    for file in os.listdir(products_folder):
        products_file = os.path.join(products_folder, file)
        if is_first:
            products = pd.read_csv(products_file, usecols=["user_id", "product_id"])
            is_first = False
        else:
            new_products = pd.read_csv(products_file, usecols=["user_id", "product_id"])
            products = pd.concat([products, new_products])

products['dummy'] = 1
data = products.groupby(["user_id", "product_id"]).agg({"dummy": "sum"})
del products
data.reset_index(inplace=True)

In [16]:
data2 = data.loc[data.user_id.isin(users_to_recommend)]
data3 = data2.sort_values("dummy", ascending=False)
data3.head()

,user_id,product_id,dummy
567820,56037,0,471
2759789,222648,3818317,364
570080,56037,6375406,360
2759213,222648,709,332
568229,56037,12565,330


In [24]:
data4 = data3.groupby(['user_id'])['product_id'] \
             .apply(lambda x: ' '.join([str(el) for el in x[:min(50, len(x))]])).reset_index()
data4.head()

,user_id,product_id
0,51,14863 63057 19562 3562687 709 9979 661 456 550...
1,65,54728 18450 3817542 709 26062 18439 105764 771...
2,187,88084 88086
3,400,0 813 94333 3817507 57085 61053 55034 26751 38...
4,576,177 2600 46469 5628 10002 102 4443 5046703 107...


In [ ]:
cosine_dummy_submission = pd.read_csv("submission_cosine_dummy.csv")

In [25]:
sub_me = []
for idx, row in data4.iterrows():
    rec = row['product_id']
    user_id = row['user_id']
    assert isinstance(rec, str)
    assert isinstance(user_id, int)
    rec_list = rec.split(' ')
    lack = 50 - len(rec_list)
    assert lack >= 0
    if lack > 0:
        cos_rec = cosine_dummy_submission.loc[cosine_dummy_submission['Id'] == user_id].Predicted.item().split(' ')
        while lack > 0:
            rec_item = cos_rec.pop(0)
            if rec_item not in rec_list:
                rec_list.append(rec_item)
                lack -= 1
        rec = ' '.join(rec_list)
    sub_me.append(rec)

In [29]:
data5 = data4.copy()
data5['product_id'] = sub_me
data5.rename(columns={"user_id": "Id", "product_id": "Predicted"}, inplace=True)
data5.head()

,Id,Predicted
0,51,14863 63057 19562 3562687 709 9979 661 456 550...
1,65,54728 18450 3817542 709 26062 18439 105764 771...
2,187,88084 88086 88085 3302679 88081 9221154 88095 ...
3,400,0 813 94333 3817507 57085 61053 55034 26751 38...
4,576,177 2600 46469 5628 10002 102 4443 5046703 107...


In [30]:
data5.to_csv("dumb_solution.csv", index=False)